# 8번
- MINIST 데이터를 훈련세트 50000개, 검증세트 10000개, 테스트 10000개로 분할
- 랜덤 포레스트, 엑스트라트리, SVM 등으로 훈련
- 직접 투표 또는 간접투표로 앙상블
- 개별 분류기와 앙상블 결과를 테스트 세트로 성능 비교
- https://github.com/rickiepark/handson-ml2/blob/master/07_ensemble_learning_and_random_forests.ipynb

##### MINIST 데이터를 훈련세트 50000개, 검증세트 10000개, 테스트 10000개로 분할

In [11]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

mnist= fetch_openml('mnist_784', version=1, as_frame=False)
X, y, X_test, y_test= mnist['data'][:60000], mnist['target'][:60000], mnist['data'][60000:], mnist['target'][60000:]

#stratify: default=None. classification을 다룰 때 성능차이가 많이 나는 옵션(데이터 편향 방지)
#stratify 값을 target으로 지정해주면 각각의 class 비율(ratio)을 train / validation에 유지. 
X_train,X_val, y_train, y_val=train_test_split(X, y, test_size=10000,stratify=y, random_state=42)

/opt/homebrew/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [12]:
print(X_test.shape, y_test.shape)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)

(10000, 784) (10000,)
(50000, 784) (50000,)
(10000, 784) (10000,)


In [13]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)

#fit_transform 학습+변환, transform 변환만
X_train= pca.fit_transform(X_train)
X_test = pca.transform(X_test)

In [14]:
print(X_test.shape, y_test.shape)
print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
#차원 축소는 언제나 훈련 시간을 줄여주지 못합니다. 데이터셋, 모델, 훈련 알고리즘에 따라 달라집니다. 

(10000, 154) (10000,)
(50000, 154) (50000,)
(10000, 784) (10000,)


##### 랜덤 포레스트, 엑스트라트리, SVM 등으로 훈련

In [27]:
from sklearn.metrics import mean_squared_error

def early_stopping(model):
    min_val_error=float('inf')
    error_goning_up= 0
    for n_estimators in range(10, 200, 10):
        print(n_estimators)
        model.n_estimators= n_estimators
        model.fit(X_train, y_train)
        #y_pred= model.predict(X_val)
        #val_error= mean_squared_error(y_val, y_pred)
        val_error= model.score(X_val, y_val)
        if val_error < min_val_error:
            min_val_error= val_error
            error_going_up= 0
        else:
            error_going_up += 1
            if error_going_up == 5:
                break
    return model

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost

rnd_clf=RandomForestClassifier(
    max_leaf_nodes=16,
    n_jobs=-1, 
    warm_start=True, 
    oob_score=True,
    bootstrap=True)
rnd_clf=early_stopping(rnd_clf)
print(rnd_clf.__class__.__name__, rnd_clf.oob_score_)

/opt/homebrew/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:584: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(
/opt/homebrew/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:584: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


RandomForestClassifier 0.8154960317460317


In [ ]:
ada_clf=AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=2),
    algorithm="SAMME.R",
    learning_rate=0.2
)
ada_clf=early_stopping(ada_clf)
print(ada_clf.__class__.__name__, ada_clf.oob_score_)

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180


In [ ]:
gb_clf=GradientBoostingClassifier(
    max_depth=2,
    warm_start=True
)
gb_clf=early_stopping(gb_clf)
print(gb_clf.__class__.__name__, gb_clf.oob_score_)

In [ ]:
xgb_clf= xgboost.XGBClassifier( 
    max_depth=2,
    n_jobs=-1, )
xgb_clf.fit(X_train, y_train, evel_set=[(X_val, y_val)], early_stopping_rounds=2)

##### 개별 성능

In [ ]:
def print_accuracy_score(model_list):
    for model in model_list:
        y_pred= model.predict(X_test)
        score= accuracy_score(y_test, y_pred)
        print(model.__class__.__name__, score)

In [ ]:
from sklearn.metrics import accuracy_score
print_accuracy_score([rnd_clf, ext_clf, ada_clf, gb_clf, xgb_clf])

##### 보팅 성능

In [ ]:
from sklearn.ensemble import VotingClassifier

vot_hard_clf= VotingClassifier(
    estimators=[("rnd_clf",rnd_clf),("ada_clf",ada_clf),("gb_clf",gb_clf),("xgb_clf",xgb_clf)],
    voting='hard'
)

vot_soft_clf= VotingClassifier(
    estimators=[("rnd_clf",rnd_clf),("ada_clf",ada_clf),("gb_clf",gb_clf),("xgb_clf",xgb_clf)],
    voting='soft'
)
print_accuracy_score([vot_hard_clf, vot_soft_clf])

# 9번
- 8번의 검증세트 결과로 새로운 훈련세트(샘플=각 분류기의 예측, 타겟=클래스) 생성
- 블렌더를 그 훈련세트로 훈련
    - MLPClassifier,RandomForestClassifier등을 블렌더 용도로 쓸 수 있다.
- 투표분류기와 블렌더의 결과 비교

In [7]:
from sklearn.ensemble import StackingClassifier
    
stk_clf = StackingClassifier(
    estimators=[rnd_clf, ext_clf, ada_clf, gb_clf, xgb_clf], 
    final_estimator=MLPClassifier()
)

print_accuracy_score([stk_clf])

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 6.4 MB/s eta 0:00:006.7 MB/s eta 0:00:01
